# IAOC24 codes for most USFS contracted aircraft

In [ ]:
# IAOC24 codes for most USFS waterbombers
IAOC24_CODES = ["C00E3E","C07CB2","A7D27A","A69072","A11CBB","A7F642","A9926A","AB79CC","A3F3AC","A3F763","A442AA","A85052",
                "A4EA6D","A380E6","A4B460","A47CBC","A48A3A","A47197","A46DE0","A4C031","A07F21",
               "A380E6","A4B50C","A0956B","A2F996","A2FD4D","A30104","A304BB","A30872","A46DE0","A47CBC","A48A3A","A47197",
                "A5CD6C","A5D123","A5D4DA","AD66E3","ADC0C4","AAFD0B","A5C9B5","A8215E","A19DA0", "A2B7FD"]

IAOC24_CODES = [code.lower() for code in IAOC24_CODES]
print(IAOC24_CODES)
print(len(IAOC24_CODES))

### ATU drop data

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString
import pandas as pd

# Paths to your shapefiles
shapefile_paths = [
    r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\2021_Full_Year_03112024_VLAT.shp",
    r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\2022_Full_Year_03112024_VLAT.shp",
    r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\2023_Full_Year_03112024_VLAT.shp"
]

# Read all shapefiles and combine them into a single GeoDataFrame
gdf_list = [gpd.read_file(shapefile_path) for shapefile_path in shapefile_paths]
combined_gdf = pd.concat(gdf_list, ignore_index=True)

# Step 2: Calculate the centroid for each line
# The centroid here refers to the geometric center of the line's bounding box, not always on the line itself
combined_gdf['centroid'] = combined_gdf.geometry.centroid

# Extract latitude and longitude from the centroid
combined_gdf['Latitude1'] = combined_gdf['centroid'].y
combined_gdf['Longitude1'] = combined_gdf['centroid'].x

# Drop the 'centroid' column as it's no longer needed (optional)
combined_gdf.drop('centroid', axis=1, inplace=True)

# Convert 'Date_(UTC)' and 'Time_(UTC)' columns to a single datetime column
combined_gdf['UTCdateTim'] = pd.to_datetime(combined_gdf['Date_(UTC)'] + ' ' + combined_gdf['Time_(UTC)'])

# Extract the prefix from 'dropID' and map to ADS-B codes
combined_gdf['dropID_prefix'] = combined_gdf['dropID'].apply(lambda x: x.split('_')[0])

# Define the mapping from dropID_prefix to ADS-B codes
dropID_to_adsb_code = {
    'T910': 'a7d27a',
    'T912': 'a5d891',
    'T911': 'a11cbb',
    'T914': 'a7d27a'
}

# Create a new column 'adsb_code' by mapping the dropID_prefix values to their corresponding ADS-B codes
combined_gdf['adsb_code'] = combined_gdf['dropID_prefix'].map(dropID_to_adsb_code)

# Print the result
#print(combined_gdf[['dropID', 'dropID_prefix', 'adsb_code', 'Latitude1', 'Longitude1', 'UTCdateTim']])

# You can also convert the GeoDataFrame to a DataFrame and save it as a CSV if needed
combined_df_vlat = combined_gdf[['dropID', 'dropID_prefix', 'adsb_code', 'Latitude1', 'Longitude1', 'UTCdateTim']]
#combined_df.to_csv('combined_vlat_data.csv', index=False)
combined_df_vlat

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString
import pandas as pd

# Paths to your shapefiles
shapefile_paths = [
    r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\LAT_20210101_20211231_FullYear.shp",
    r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\LAT_20220101_20221231_FullYear.shp",
    r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\LAT_20230101_20231231_FullYear.shp"
]

# Read the target CRS from one of the shapefiles
shapefile_path_2 = r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\2023_Full_Year_03112024_VLAT.shp"
gdf_2 = gpd.read_file(shapefile_path_2)
target_crs = gdf_2.crs
print(f"Target CRS: {target_crs}")

# Process each shapefile
gdf_list = []
for shapefile_path in shapefile_paths:
    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.to_crs(target_crs)
    print(f"Original length of {shapefile_path}: {len(gdf)}")
    
    # Filter by LineLenMi
    gdf = gdf[gdf['LineLenMi'] < 0.2]
    print(f"Filtered length of {shapefile_path}: {len(gdf)}")
    
    # Calculate the centroid for each line
    gdf['centroid'] = gdf.geometry.centroid
    
    # Extract latitude and longitude from the centroid
    gdf['Latitude1'] = gdf['centroid'].y
    gdf['Longitude1'] = gdf['centroid'].x
    
    # Drop the 'centroid' column as it's no longer needed (optional)
    gdf.drop('centroid', axis=1, inplace=True)
    
    # Convert date-time column
    gdf['UTCdateTim'] = pd.to_datetime(gdf['DateTimeUT'])
    gdf['dropID_prefix'] = gdf['DropID'].apply(lambda x: x.split('_')[0])
    
    gdf_list.append(gdf)

# Combine all the GeoDataFrames
combined_gdf = pd.concat(gdf_list, ignore_index=True)
combined_gdf.rename(columns={'n': 'TailNumber'}, inplace=True)

# Your mapping of 'n' values to 'adsb_CODE' values
n_to_adsb_code = {
    'N389AC': 'A47CBC',
    'N392AC': 'A48A3A',
    'N386AC': 'A47197',
    'N385AC': 'A46DE0',
    'N473NA': 'A5CD6C',
    'N474NA': 'A5D123',
    'N475NA': 'A5D4DA',
    'N472NA': 'A5C9B5',
    'N291EA': 'A2F996',
    'N292EA': 'A2FD4D',
    'N293EA': 'A30104',
    'N294EA': 'A304BB',
    'N295EA': 'A30872',
    'N297EA': 'A30FE0',
    'N476NA': 'A5D891',
    'N131CG': 'A07F21',
    'N132CG': 'A082D8',
    'N137CG': 'A0956B',
    'N477NA': 'A5DC48',
    'C-FKFM': 'CFKFM',
    'N478NA': 'A5DFFF',
    'N839AC': 'AB79CC',
    'N355AC': 'A3F763',
    'N366AC': 'A42299',
    'N374AC': 'A442AA',
    'N635AC': 'A85052',
    'N416AC': 'A4EA6D',
    'N325AC': 'A380E6',
    'N138CG': 'A09922',
    'N470NA': 'A5C247',
    'N471NA': 'A5C5FE',
    'C-FFQF': 'CFFQF',
    'C-FFQG': 'CFFQG',
    'N358AS': 'A40296',
    'N23WT': 'A2082D',
    'N90WW': 'AC6EDC',
    'N10TP': 'A00412',
    'N619SW': 'A811D2',
    'N415BT': 'A4E6DE',
    'N417BT': 'A4EE4C',
    'N418BT': 'A4F203',
    'N419BT': 'A4F5BA',
    'N406BT': 'A4C316',
    'C-FFQL': 'CFFQL',
    'C-FFZJ': 'CFFZJ',
}

#['N619SW', 'N415BT', 'N417BT', 'N418BT', 'N419BT', 'N406BT', 'C-FFQL', 'C-FFZJ']


combined_gdf['adsb_CODE_UPPER'] = combined_gdf['TailNumber'].map(n_to_adsb_code)

# Convert 'adsb_CODE' values to lowercase, safely handling NaN values
#data['adsb_CODE_lower'] = [code.lower() if pd.notnull(code) else None for code in data['adsb_CODE']]
combined_gdf['adsb_code'] = combined_gdf['adsb_CODE_UPPER'].str.lower()
combined_gdf['UTCdateTim']= pd.to_datetime(combined_gdf['UTCdateTim'])

# If you want to create a list of these lowercase 'adsb_CODE' values (excluding None)
#dataADSBLIST = [code for code in data['adsb_CODE_lower'] if code is not None]



# Output the combined GeoDataFrame
#print(combined_gdf[['DropID', 'dropID_prefix', 'Latitude1', 'Longitude1', 'UTCdateTim']])

# Optionally, save the combined GeoDataFrame to a new shapefile or CSV
#combined_gdf.to_file("combined_vlat_data.shp")
#combined_gdf[['DropID', 'dropID_prefix', 'Latitude1', 'Longitude1', 'UTCdateTim']].to_csv("combined_vlat_data.csv", index=False)


In [ ]:
combined_gdf

In [ ]:
import os
import pandas as pd

# Load the data
data = pd.read_csv(r"C:\Users\admin-magstadt\Desktop\ADSBflighttracking\ATU_2017_2021_1mile_ALL.txt", sep=",", header=0)

# Only considering the first 5000 rows as per your previous request
#data = data.iloc[0:5000]

# Replace ':' with '_' in 'Drop_ID1' column
data['Drop_ID1'] = data['Drop_ID1'].str.replace(':', '_')
data['Drop_ID1'] = data['Drop_ID1'].str.replace('-', '_')
data['Drop_ID1'] = data['Drop_ID1'].str.replace(' ', '')
data.rename(columns={'Drop_ID1': 'DropID'}, inplace=True)
data.rename(columns={'Tail_Numbe': 'TailNumber'}, inplace=True)

# List all filenames in the specified directory (without file extensions)
# directory_path = r"C:\Users\admin-magstadt\Desktop\ADSBData\DropSamplesADSB"
# filenames = [os.path.splitext(file)[0] for file in os.listdir(directory_path)]

# # Check if modified 'Drop_ID1' values are in the list of filenames
# data['FilenameMatch'] = data['Drop_ID1'].isin(filenames)

# # Filter the DataFrame to only include rows where 'Drop_ID1' matches a filename
# data = data[data['FilenameMatch']]

# # Drop the 'FilenameMatch' column if you don't need it anymore
# data.drop(columns=['FilenameMatch'], inplace=True)

# print("done")




def map_tail_number_to_adsb_code(tail_number):
    if tail_number == "T-01":
        return "a5cd6c"
    elif tail_number == "T-02":
        return "a5d123"
    elif tail_number == "T-03":
        return "a5d4da"
    elif tail_number == "T-05":
        return "ad66e3"
    elif tail_number == "T-06":
        return "adc0c4"
    elif tail_number == "T-07":
        return "aafd0b"
    elif tail_number == "T-10":
        return "a5c9b5"
    elif tail_number == "T-101":
        return "a2f996"
    elif tail_number == "T-102":
        return "a2fd4d"
    elif tail_number == "T-103":
        return "a30104"
    elif tail_number == "T-104":
        return "a304bb"
    elif tail_number == "T-105":
        return "a30872"
    elif tail_number == "T-107":
        return "a30fe0"
    elif tail_number == "T-12":
        return "a5d891"
    elif tail_number == "T-131":
        return "a07f21"
    elif tail_number == "T-132":
        return "a4c031"
    elif tail_number == "T-133":
        return "a4b50c"
    elif tail_number == "T-137":
        return "a0956b"
    elif tail_number == "T-14":
        return "a8215e"
    elif tail_number == "T-142":
        return "a19da0"
    elif tail_number == "T-15":
        return "a2b7fd"
    elif tail_number == "T-152":
        return "ac75f9"
    elif tail_number == "T-16":
        return "a5dfff"
    elif tail_number == "T-160":
        return "ab79cc"
    elif tail_number == "T-161":
        return "a3f3ac"
    elif tail_number == "T-162":
        return "a3f763"
    elif tail_number == "T-163":
        return "a42299"
    elif tail_number == "T-164":
        return "a442aa"
    elif tail_number == "T-166":
        return "c07cb2"
    elif tail_number == "T-167":
        return "a85052"
    elif tail_number == "T-168":
        return "a4ea6d"
    elif tail_number == "T-169":
        return "a380e6"
    elif tail_number == "T-210":
        return "a4b460"
    elif tail_number == "T-260":
        return "a47cbc"
    elif tail_number == "T-261":
        return "a48a3a"
    elif tail_number == "T-262":
        return "a47197"
    elif tail_number == "T-263":
        return "a46de0"
    elif tail_number == "T-40":
        return "A9A086"
    elif tail_number == "T-41":
        return "NA"
    elif tail_number == "T-44":
        return "c00e3e"
    elif tail_number == "T-910":
        return "a7f642"
    elif tail_number == "T-911":
        return "a11cbb"
    elif tail_number == "T-912":
        return "a69072"
    elif tail_number == "T-914":
        return "a7d27a"
    elif tail_number == "T-944":
        return "NA"
    elif tail_number == "N130CG":
        return "a07b6a"
    else:    
        return "UNKNOWN_ADSB_CODE"

    
# Create a new column "adsbCODE" based on the values in the "Tail_Numbe" column
data["adsb_code"] = data["TailNumber"].apply(lambda x: map_tail_number_to_adsb_code(x))
data.UTCdateTim = data.UTCdateTim.str[:-4]
data['UTCdateTim']= pd.to_datetime(data['UTCdateTim'])


In [ ]:
import pandas as pd

# Assuming the DataFrames are already loaded as data, combined_gdf, and combined_df_vlat

# Select the relevant columns from each DataFrame
df1_selected = data[['DropID', 'Latitude1', 'Longitude1', 'UTCdateTim', 'TailNumber', 'adsb_code']]
df2_selected = combined_gdf[['DropID', 'Latitude1', 'Longitude1', 'UTCdateTim','TailNumber', 'adsb_code']]
df3_selected = combined_df_vlat[['dropID', 'Latitude1', 'Longitude1', 'UTCdateTim', 'adsb_code']]

# Rename the 'dropID' column in df3_selected to 'DropID' for consistency
#df3_selected.rename(columns={'dropID': 'DropID'}, inplace=True)

# Concatenate the DataFrames
combined_df = pd.concat([df1_selected, df2_selected])

# Reset the index if necessary
combined_df.reset_index(drop=True, inplace=True)
# Display the combined DataFrame
print(combined_df)


In [ ]:
combined_df

In [ ]:
#combined_df.to_csv('C:\\Users\\admin-magstadt\\Desktop\\Output\\ATUdropData_2017_2023.csv', sep=',', index=False)


### Function to map tail numbers to ADSB codes

In [ ]:
# def map_tail_number_to_adsb_code(tail_number):
#     if tail_number == "T-01":
#         return "a5cd6c"
#     elif tail_number == "T-02":
#         return "a5d123"
#     elif tail_number == "T-03":
#         return "a5d4da"
#     elif tail_number == "T-05":
#         return "ad66e3"
#     elif tail_number == "T-06":
#         return "adc0c4"
#     elif tail_number == "T-07":
#         return "aafd0b"
#     elif tail_number == "T-10":
#         return "a5c9b5"
#     elif tail_number == "T-101":
#         return "a2f996"
#     elif tail_number == "T-102":
#         return "a2fd4d"
#     elif tail_number == "T-103":
#         return "a30104"
#     elif tail_number == "T-104":
#         return "a304bb"
#     elif tail_number == "T-105":
#         return "a30872"
#     elif tail_number == "T-107":
#         return "a30fe0"
#     elif tail_number == "T-12":
#         return "a5d891"
#     elif tail_number == "T-131":
#         return "a07f21"
#     elif tail_number == "T-132":
#         return "a4c031"
#     elif tail_number == "T-133":
#         return "a4b50c"
#     elif tail_number == "T-137":
#         return "a0956b"
#     elif tail_number == "T-14":
#         return "a8215e"
#     elif tail_number == "T-142":
#         return "a19da0"
#     elif tail_number == "T-15":
#         return "a2b7fd"
#     elif tail_number == "T-152":
#         return "ac75f9"
#     elif tail_number == "T-16":
#         return "a5dfff"
#     elif tail_number == "T-160":
#         return "ab79cc"
#     elif tail_number == "T-161":
#         return "a3f3ac"
#     elif tail_number == "T-162":
#         return "a3f763"
#     elif tail_number == "T-163":
#         return "a42299"
#     elif tail_number == "T-164":
#         return "a442aa"
#     elif tail_number == "T-166":
#         return "c07cb2"
#     elif tail_number == "T-167":
#         return "a85052"
#     elif tail_number == "T-168":
#         return "a4ea6d"
#     elif tail_number == "T-169":
#         return "a380e6"
#     elif tail_number == "T-210":
#         return "a4b460"
#     elif tail_number == "T-260":
#         return "a47cbc"
#     elif tail_number == "T-261":
#         return "a48a3a"
#     elif tail_number == "T-262":
#         return "a47197"
#     elif tail_number == "T-263":
#         return "a46de0"
#     elif tail_number == "T-40":
#         return "A9A086"
#     elif tail_number == "T-41":
#         return "NA"
#     elif tail_number == "T-44":
#         return "c00e3e"
#     elif tail_number == "T-910":
#         return "a7f642"
#     elif tail_number == "T-911":
#         return "a11cbb"
#     elif tail_number == "T-912":
#         return "a69072"
#     elif tail_number == "T-914":
#         return "a7d27a"
#     elif tail_number == "T-944":
#         return "NA"
#     elif tail_number == "N130CG":
#         return "a07b6a"
#     else:    
#         return "UNKNOWN_ADSB_CODE"

    
# # Create a new column "adsbCODE" based on the values in the "Tail_Numbe" column
# combined_df["adsbCODE"] = combined_df["TailNumber"].apply(lambda x: map_tail_number_to_adsb_code(x))


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
file_path = 'C:\\Users\\admin-magstadt\\Desktop\\Output\\ATUdropData_2017_2023.csv'
df = pd.read_csv(file_path, sep=',')

# Display the DataFrame
print(df)


In [ ]:
data

### Create columns to define the start and end of the query (identify the space-time bounding box)

In [ ]:
from datetime import datetime
from datetime import datetime, timedelta
import pandas as pd
import pandas as pd
import numpy as np


#data.UTCdateTim = data.UTCdateTim.str[:-4]

#print(data.UTCdateTim)
#data['UTCdateTim']= pd.to_datetime(data['UTCdateTim'])

# isolates the drop time - 30 second window
time_change = timedelta(minutes=0.25)
data["Time2"] = data.UTCdateTim + time_change
data["Time1"] = data.UTCdateTim - time_change

time_change2 = timedelta(minutes=720)
data["Time2_2"] = data.UTCdateTim + time_change2
data["Time1_2"] = data.UTCdateTim - time_change2

# define bounding box
data["Latitude1PLUS"] = data.Latitude1+0.02
data["Latitude1MINUS"] = data.Latitude1-0.02

data["Longitude1PLUS"] = data.Longitude1+0.02
data["Longitude1MINUS"] = data.Longitude1-0.02

dataLatitude1PLUS =data["Latitude1PLUS"].tolist()
dataLatitude1MINUS =data["Latitude1MINUS"].tolist()

dataLongitude1PLUS =data["Longitude1PLUS"].tolist()
dataLongitude1MINUS =data["Longitude1MINUS"].tolist()

dataLongitude1 =  data.Longitude1.tolist()
dataLatitude1 = data.Latitude1.tolist()

# 15 seconds prior to and post start of drop
datetime1 = data.Time1.tolist()
datetime2 = data.Time2.tolist()

# 3 minutes prior to and after the start of the drop (for non-drop samples)
datetime1_2 = data.Time1_2.tolist()
datetime2_2 = data.Time2_2.tolist()


data=data.replace(regex=[':'], value='_')
data=data.replace(regex=['-'], value='_')
data=data.replace(regex=[' '], value='')
data_DropID = data.DropID.tolist()

print(len(data_DropID))
print("done")

In [ ]:
from pyopensky import OpenskyImpalaWrapper
from osgeo import ogr
from osgeo import osr
import csv

OutputDirectoryDropSamples = "C:\\Users\\admin-magstadt\\Desktop\\VLATfromJIM\\Output\\OutputDropSamples2\\"

OutputDirectoryNonDropSamples = "C:\\Users\\admin-magstadt\\Desktop\\VLATfromJIM\\Output\\OutputNonDropSamples2\\"

opensky = OpenskyImpalaWrapper()

for i in range(len(data)):

    print(i)
    
    # Perform a query with ICAO filter
    df = opensky.query(
        type="adsb",
        #start = datetime1[i],
        #end = datetime2[i],
        start = datetime1_utc[i],
        end = datetime2_utc[i],
        bound=[dataLatitude1MINUS[i],dataLongitude1MINUS[i],dataLatitude1PLUS[i],dataLongitude1PLUS[i]],
        #icao24 = adsbCODES
        icao24 = dataADSBLIST[i]
        #icao24 = adsbCODES[i]
        #icao24 = IAOC24_CODES
    )
    
    if df is not None:        
        # this filters the data and ensure there is continuous quality data. The main issue I was facing was limited data availability, 
        # particularly in low elevation mountainous regions with limited ADSB coverage by OpenSky
        # this will drastically limit the avalible data
        if (df['lastposupdate'].nunique() > (len(df)-15)) is True and (df['icao24'].nunique() == 1) is True and (len(df) > 25) is True:
        #if (len(df) > 25) is True:

            icaocode24 = df['icao24']
            print("itsUniqueEnough")
            df2 = df.dropna(subset = ["lat"])          # Apply dropna() function to remove missing lat lon 
            if len(df2) > 0:
                filepathcsv = OutputDirectoryDropSamples+data_DropID[i]+".csv"
                df2.to_csv(filepathcsv)
                filepathshp = OutputDirectoryDropSamples+data_DropID[i]+".shp"
                driver = ogr.GetDriverByName("ESRI Shapefile")
                data_src = driver.CreateDataSource(filepathshp)
                srs = osr.SpatialReference()
                srs.ImportFromEPSG(4326)# 4326 = wgs84
                layer = data_src.CreateLayer(filepathshp, 
                                             srs, 
                                             geom_type = ogr.wkbPoint)

                #Create attribute fields from OpenSky
                field_name = ogr.FieldDefn("time", ogr.OFTString)
                field_name.SetWidth(50)
                layer.CreateField(field_name)
                layer.CreateField(ogr.FieldDefn("lon", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("lat", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("velocity", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("heading", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("vertrate", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("callsign", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("onground", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("alert", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("spi", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("squawk", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("baro", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("geo", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("lastpos", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("lastcon", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("hour", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("icao24", ogr.OFTString))
  
                with open(filepathcsv, "r") as csv_file:
                    csv_reader = csv.reader(csv_file)
                    next(csv_reader)
                    for row in csv_reader:        
                        feature = ogr.Feature(layer.GetLayerDefn())
                        feature.SetField("time", row[1])
                        feature.SetField("lon", row[4])
                        feature.SetField("lat", row[3])
                        feature.SetField("velocity", row[5])
                        feature.SetField("heading", row[6])
                        feature.SetField("vertrate", row[7])
                        feature.SetField("callsign", row[8])
                        feature.SetField("onground", row[9])
                        feature.SetField("alert", row[10])
                        feature.SetField("spi", row[11])
                        feature.SetField("squawk", row[12])
                        feature.SetField("baro", row[13])
                        feature.SetField("geo", row[14])
                        feature.SetField("lastpos", row[15])
                        feature.SetField("lastcon", row[16])
                        feature.SetField("hour", row[17])
                        feature.SetField("icao24", row[2])

                        #Create point geometry
                        point = ogr.Geometry(ogr.wkbPoint)
                        point.AddPoint(float(row[4]), float(row[3]))

                        #Create the feature and set the values 
                        feature.SetGeometry(point)
                        layer.CreateFeature(feature)
                        # reset features for next row
                        feature = None
                data_src = None
            
            ##
            dfLong = opensky.query(
                type="adsb",
                start = datetime1_2_utc[i],
                end = datetime2_2_utc[i],
                icao24 = icaocode24
                )
            
            df2_long = dfLong.dropna(subset = ["lat"])          # Apply dropna() function to remove missing lat lon 
            if len(df2_long) > 0:
                filepathcsv = OutputDirectoryNonDropSamples+data_DropID[i]+".csv"
                df2_long.to_csv(filepathcsv)
                filepathshp = OutputDirectoryNonDropSamples+data_DropID[i]+".shp"
                driver = ogr.GetDriverByName("ESRI Shapefile")
                data_src = driver.CreateDataSource(filepathshp)
                srs = osr.SpatialReference()
                srs.ImportFromEPSG(4326)# 4326 = wgs84
                layer = data_src.CreateLayer(filepathshp, 
                                             srs, 
                                             geom_type = ogr.wkbPoint)

                #Create attribute fields from OpenSky
                field_name = ogr.FieldDefn("time", ogr.OFTString)
                field_name.SetWidth(50)
                layer.CreateField(field_name)
                layer.CreateField(ogr.FieldDefn("lon", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("lat", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("velocity", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("heading", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("vertrate", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("callsign", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("onground", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("alert", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("spi", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("squawk", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("baro", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("geo", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("lastpos", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("lastcon", ogr.OFTReal))
                layer.CreateField(ogr.FieldDefn("hour", ogr.OFTString))
                layer.CreateField(ogr.FieldDefn("icao24", ogr.OFTString))

                with open(filepathcsv, "r") as csv_file:
                    csv_reader = csv.reader(csv_file)
                    next(csv_reader)
                    for row in csv_reader:        
                        feature = ogr.Feature(layer.GetLayerDefn())
                        feature.SetField("time", row[1])
                        feature.SetField("lon", row[4])
                        feature.SetField("lat", row[3])
                        feature.SetField("velocity", row[5])
                        feature.SetField("heading", row[6])
                        feature.SetField("vertrate", row[7])
                        feature.SetField("callsign", row[8])
                        feature.SetField("onground", row[9])
                        feature.SetField("alert", row[10])
                        feature.SetField("spi", row[11])
                        feature.SetField("squawk", row[12])
                        feature.SetField("baro", row[13])
                        feature.SetField("geo", row[14])
                        feature.SetField("lastpos", row[15])
                        feature.SetField("lastcon", row[16])
                        feature.SetField("hour", row[17])
                        feature.SetField("icao24", row[2])


                        #Create point geometry
                        point = ogr.Geometry(ogr.wkbPoint)
                        point.AddPoint(float(row[4]), float(row[3]))

                        #Create the feature and set the values 
                        feature.SetGeometry(point)
                        layer.CreateFeature(feature)
                        # reset features for next row
                        feature = None
                data_src = None


# Calculate HeightAGL

In [1]:
import os
import ee
import geopandas as gpd

#ee.Authenticate()

#ee.Initialize(project = 'ee-magstadt')
ee.Initialize()


#input_folder = pathshp
#input_folder = "C:\\Users\\admin-magstadt\\Desktop\\VLATfromJIM\\Output\\OutputDropSamples\\"
input_folder = "C:\\Users\\admin-magstadt\\Desktop\\DropSamples\\"
#output_folder_short = "C:\\Users\\admin-magstadt\\Desktop\\VLATfromJIM\\Output\\OutputNonDropSamplesWAGL\\"
#output_folder_short = "C:\\Users\\admin-magstadt\\Desktop\\CLEANATUDROPSAMPLES\\"
output_folder_short = "C:\\Users\\admin-magstadt\\Desktop\\DropSamplesAGL\\"

desired_columns_order = ['heading', 'icao24', 'lon', 'onground', 'velocity', 'lastpos', 'spi', 'geo', 'vertrate', 'baro', 'squawk', 'hour', 'alert', 'callsign', 'time', 'lastcon', 'lat', 'elevation', 'slope', 'hillshade', 'aspect', 'heightAGL']


# Loop over all shapefiles in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".shp"):
        #print("Processing:", filename)
        # Construct input and output file paths
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder_short, filename[:-4] + ".csv")  # Remove ".shp" from input filename and add ".csv"
        
        # Run the existing script with the input and output file paths
        import ee
        import geemap
        import pandas as pd
        import time

        try:
            start_time = time.time()

            ee_fc = geemap.shp_to_ee(input_path)
            buffered_geometry = ee_fc.geometry().bounds().buffer(1000)  # Adjust buffer distance as needed

#             # NEW
#             df = pd.read_csv(input_path)

#             # Function to create a feature from a row
#             def row_to_feature(row):
#                 geom = ee.Geometry.Point([row['lon'], row['lat']])
#                 feature = ee.Feature(geom, row.to_dict())
#                 return feature

#             # Create a list of ee.Feature objects
#             features = df.apply(row_to_feature, axis=1).tolist()

#             # Convert the list of features into a FeatureCollection
#             ee_feature_collection = ee.FeatureCollection(features)
#             # end new
           
            #Print the number of features in the FeatureCollection
            num_features = ee_fc.size().getInfo()
            #print(f"Number of features in {filename}: {num_features}")
#             shapefile = gpd.read_file(input_path)
#             ee_fc = ee.Geometry.MultiPolygon(shapefile)#.geometry.to_crs(epsg='4326').map(lambda x: x.__geo_interface__['coordinates']))

            
            
             # Load the 3DEP dataset  and others and filter by the extent of the points
            dem = ee.Image("USGS/3DEP/10m").clip(buffered_geometry)#ee_fc.geometry().bounds())
            
            #dem = dem.where(dem.mask().Not(), 0)
        
            # Extract data for each image at appropriate scale
            terrain_fc = ee.Terrain.products(dem).sampleRegions(collection=ee_fc, scale=10)#, geometries=False)
            #terrain_fc_size = terrain_fc.size()

            # Fetch the size (number of features) as a client-side value
            #print(terrain_fc_size.getInfo(), "ddd")

            # Convert the elevation and terrain product data to Pandas DataFrames
            terrain_df = geemap.ee_to_pandas(terrain_fc)
            #print(len(terrain_df), "ss")  # Print the columns to check the actual column names
            
            #print(terrain_df)

            # Drop rows with missing values
            df_sample = terrain_df#.dropna()
            #df_sample = terrain_df.fillna(method='bfill').dropna()
            
            #print(len(df_sample))  # Print the columns to check the actual column names

            # Calculate height AGL
            baro_altitude = df_sample['geo']
            elevation = df_sample['elevation']
            height_AGL = baro_altitude - elevation
            df_sample['heightAGL'] = height_AGL
            
            df_sample = df_sample.reindex(columns=desired_columns_order)

            # Export as CSV
            #df_sample.head(30).to_csv(output_path, index=False)
            df_sample.to_csv(output_path, index=False)

            #print("Completed:", filename)
        except Exception as e:
            print("Error occurred:", e)
            continue
print("DONE")

C:\Users\admin-magstadt\anaconda3\envs\ee\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


Error occurred: 'geo'
Error occurred: 'geo'
Error occurred: 'geo'
Error occurred: 'geo'
Error occurred: 'geo'
Error occurred: 'geo'
Error occurred: Invalid JSON payload received. Unexpected token.
: {"constantValue": NaN}, "vertical_r": 
                    ^
DONE


In [ ]:
# Clean up data 

In [ ]:
import pandas as pd

#COMBINE ALLDROPS
import pandas as pd
import os

# Define the path to the folder containing CSV files
folder_path = r"C:\Users\admin-magstadt\Desktop\CLEANATUDROPDATA"

# Initialize an empty list to store DataFrames from each CSV file
dfs = []

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        # Read each CSV file into a DataFrame and append it to the list
        dfs.append(pd.read_csv(file_path))

# Concatenate all DataFrames in the list to create one big DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame
#print(combined_df)



# Define the path to the folder containing CSV files
folder_path = r"C:\Users\admin-magstadt\Desktop\VLATfromJIM\Output\OutputNonDropSamples"

# Initialize an empty list to store DataFrames from each CSV file
dfs = []

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        # Read each CSV file into a DataFrame and append it to the list
        dfs.append(pd.read_csv(file_path))

# Concatenate all DataFrames in the list to create one big DataFrame
combined_df_nondrop = pd.concat(dfs, ignore_index=True)

print(len(combined_df_nondrop))

# Display the combined DataFrame
combined_df_nondrop

combined_df_nondrop_noDup = combined_df_nondrop.drop_duplicates()

print(len(combined_df_nondrop_noDup))

print("done")






import pandas as pd
import os

df_non_drop_samples = combined_df_nondrop_noDup

# Extract values from the "time" column in the second DataFrame
drop_samples_times = combined_df["time"].tolist()
drop_samples_icao24 = combined_df["icao24"].tolist()

# Filter the first DataFrame to exclude rows where the "time" column matches values from the second DataFrame
filtered_df_non_drop_samples = df_non_drop_samples[~df_non_drop_samples[['time', 'icao24']].apply(tuple, axis=1).isin(zip(drop_samples_times, drop_samples_icao24))]

# Calculate the number of chunks
num_chunks = len(filtered_df_non_drop_samples) // 30
print(num_chunks)
# Iterate through chunks and export as CSV
for i in range(num_chunks):
    chunk = filtered_df_non_drop_samples.iloc[i*30:(i+1)*30]
    # Check if the number of unique times in the chunk is 30
    time_diff = (chunk['time'].max()+1) - chunk['time'].min()
    unique_times_count = chunk['lastcontact'].nunique()
    unique_ADSB_count = chunk['icao24'].nunique()

    if time_diff == 30 and unique_times_count > 26 and unique_ADSB_count==1:
        file_name = "combined_df_nondrop_noDup_chunk"  # Change the file name as needed
        chunk.to_csv(f"C:\\Users\\admin-magstadt\\Desktop\\TESTAGL\\{file_name}_{i+1}.csv", index=False)
        
        gdf = gpd.GeoDataFrame(chunk, geometry=gpd.points_from_xy(chunk['lon'], chunk['lat']))
        gdf.crs = 'epsg:4326'  # Set the CRS as needed
        gdf.to_file(f"C:\\Users\\admin-magstadt\\Desktop\\TESTAGL\\{file_name}_{i+1}.shp")      
        
    else:
        print(f"Skipping chunk {i+1} as it does not have 30 unique times.")


# Prepare drop and non drop samples

In [ ]:

import pandas as pd
import numpy as np
import glob

#dir_path_dropsamples = "C:\\Users\\admin-magstadt\\Desktop\\ADSBData2\\DropSamplesADSB\\"
#dir_path_dropsamples = "C:\\Users\\admin-magstadt\\Desktop\\ADSBData\\SmoothedDropSamples\\"
#dir_path_dropsamples = "C:\\Users\\admin-magstadt\\Desktop\\CLEANATUDROPDATA\\"
#dir_path_dropsamples = "C:\\Users\\admin-magstadt\\Desktop\\TEST\\DROP\\"
dir_path_dropsamples = r"C:\\Users\\admin-magstadt\\Desktop\\CLEANATUDROPSAMPLES\\"


filenames = glob.glob(dir_path_dropsamples + "*.csv")
data = []
usecols = [8,21]  # Specified columns to use

#usecols = [4,22,21]  # Specified columns to use
#usecols = [4,8]  # Specified columns to use

for filename in filenames:
    num_rows = sum(1 for line in open(filename))
    if num_rows < 26:
        continue
    skip_rows = max(0, num_rows - 30)  # Adjust to ensure reading up to the last 30 rows
    df = pd.read_csv(filename, usecols=usecols, skiprows=skip_rows)
    #df = df.dropna()
    arr = df.values.astype(float)
    # Check if the array has fewer rows than expected and pad if necessary
    if arr.shape[0] < 30:
        pad_size = 30 - arr.shape[0]
        # Use the last row of arr for padding
        last_row = arr[-1:]
        padding = np.repeat(last_row, pad_size, axis=0)
        arr = np.vstack([padding, arr])  # Prepend the padding
    data.append(arr)

if len(data) == 0:
    print("No data found")
else:
    array1 = np.stack(data, axis=0)
    #print(array1.shape)
    array1 = np.stack(data, axis=0)
    print(array1.shape)

    # Check for NaN values
    nan_indices = np.argwhere(np.isnan(array1))
    if len(nan_indices) > 0:
        print("NaN values found at indices:", nan_indices)
    else:
        print("No NaN values found")


In [ ]:
import pandas as pd
import numpy as np
import glob

dir_path_dropsamples = r"C:\Users\admin-magstadt\Desktop\CLEANATUNONDROPSAMPELS\\"
filenames = glob.glob(dir_path_dropsamples + "*.csv")

data = []


for filename in filenames:
    num_rows = sum(1 for line in open(filename))
    if num_rows < 26:
        continue
    skip_rows = max(0, num_rows - 30)
    df = pd.read_csv(filename, usecols=usecols, skiprows=skip_rows)
    
    # Drop rows with any NaN values
    df = df.dropna()
    
    arr = df.values.astype(float)
    
    if arr.shape[0] < 30:
        pad_size = 30 - arr.shape[0]
        last_row = arr[-1:]
        padding = np.repeat(last_row, pad_size, axis=0)
        arr = np.vstack([padding, arr])
    
    data.append(arr)

if len(data) == 0:
    print("No data found")
else:
    array2 = np.stack(data, axis=0)
    print(array2.shape)

    nan_indices = np.argwhere(np.isnan(array2))
    if len(nan_indices) > 0:
        print("NaN values found at indices:", nan_indices)
    else:
        print("No NaN values found")
